In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress as cmap
from scipy.stats import spearmanr as scor
from scipy.stats import mannwhitneyu as mwu

# Downloading LINCS data
From Gene Expression Omnibus downloading the [GSE92742](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92742) and the [GSE70138](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70138) datasets. From both datasets I dowloaded the **Level5 gene expression profiles**, the metadata from the **sig_info** files and a **gene info** file.\
I downloaded the **drug information** and the **sample information** for  compound metadata from the [Drug Repurposing Hub](https://clue.io/repurposing)

In [2]:
#checking if the files are in the correct directory
import os
files_needed_GSE92742=['GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
               'GSE92742_Broad_LINCS_sig_info.txt','GSE92742_Broad_LINCS_gene_info.txt']
files_needed_GSE70138=['GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                      'GSE70138_Broad_LINCS_sig_info.txt']
files_needed_rep=['repurposing_drugs_20180907.txt','repurposing_samples_20180907.txt']
files_GSE92742=os.listdir('../data/GSE92742/')
files_GSE70138=os.listdir('../data/GSE70138/')
files_rep=os.listdir('../data/repurposing/')
for f in files_needed_GSE92742:
    assert (f in files_GSE92742)
for f in files_needed_GSE70138:
    assert (f in files_GSE70138)
for f in files_needed_rep:
    assert (f in files_rep)

**Importing gene ids** 

In [3]:
gene_ids=pd.read_csv('../data/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',
                    sep='\t',header=0,index_col=0)
gene_ids.head()

,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing
pr_gene_id,,,,
780,DDR1,discoidin domain receptor tyrosine kinase 1,1,1
7849,PAX8,paired box 8,1,1
2978,GUCA1A,guanylate cyclase activator 1A,0,0
2049,EPHB3,EPH receptor B3,0,1
2101,ESRRA,estrogen related receptor alpha,0,1


In [4]:
fil=gene_ids['pr_is_lm']==1
gene_ids=gene_ids[fil]
gene_ids=gene_ids['pr_gene_symbol']
gene_ids.head()

pr_gene_id
780      DDR1
7849     PAX8
6193     RPS5
23      ABCF1
9552    SPAG7
Name: pr_gene_symbol, dtype: object

In [5]:
gene_ids.index=gene_ids.index.astype(str)

In [6]:
#To calculate consensus signature,I used the MODZ method
#described in the original LINCS manuscript.
def calc_MODZ(data):
    if len(data)==1:
        return data.iloc[0]
    if len(data)==2:
        return np.mean(data,0)
    else:
        CM=scor(data.T)[0]
        fil=CM<0
        CM[fil]=0.01
        weights=np.sum(CM,1)-1
        weights=weights/np.sum(weights)
        weights=weights.reshape((-1,1))
        return pd.Series(np.dot(data.T,weights).reshape((-1,1)[0]),index=data.columns)

# Importing ligand-receptor network 
I downloaded the receptor-ligand network from [here](https://zenodo.org/record/3260758/files/lr_network.rds), and preprocessed it in R. From it I extracted the **ligand** and **receptor** genes

In [7]:
#column 'to': receptors, column 'from': ligands
ligand_receptor=pd.read_csv('../data/lr_network.csv', sep=',', header=0, index_col=0)

In [8]:
good_sources=['kegg_cytokines', 'kegg_neuroactive','pharmacology', 'ramilowski_known']

In [9]:
fil=np.in1d(ligand_receptor['source'], good_sources)
ligand_receptor=ligand_receptor[fil]

In [10]:
receptors=ligand_receptor['to'].unique()
ligands=ligand_receptor['from'].unique()

In [11]:
l_r=list(receptors)+list(ligands)

# Getting the drugs from the Drug Repurposing Hub

In [12]:
drugs=pd.read_csv('../data/repurposing/repurposing_drugs_20180907.txt', sep='\t', index_col=None, header=0, encoding='latin', skiprows=9)

In [13]:
fil=drugs['target'].isnull()
drugs=drugs[~fil]

In [14]:
drugs.head()

,pert_iname,clinical_phase,moa,target,disease_area,indication
0,"[sar9,met(o2)11]-substance-p",Preclinical,tachykinin antagonist,TACR1,NaN,NaN
1,A-1070722,Preclinical,glycogen synthase kinase inhibitor,GSK3A|GSK3B,NaN,NaN
2,A-1120,Preclinical,retinoid receptor ligand,RBP4,NaN,NaN
3,A-317491,Preclinical,purinergic receptor antagonist,P2RX3,NaN,NaN
5,A-366,Preclinical,histone lysine methyltransferase inhibitor,EHMT1|EHMT2,NaN,NaN


In [15]:
#creating a file that has the drug, its target and the target type
def split_dataframe(one_line):
    if '|' in one_line['target']:
        targets=one_line['target'].split('|')
        temp=pd.DataFrame(index=range(len(targets)), columns=one_line.index)
        for col in temp.columns:
            temp[col]=one_line[col]
        temp['target']=targets
        return temp
    else:
        return pd.DataFrame(one_line).T

In [16]:
results=pd.DataFrame(columns=drugs.columns)
for i in drugs.index:
    one_line=drugs.loc[i]
    results=pd.concat([results, split_dataframe(one_line)])

In [17]:
#results.to_csv('../results/lincs_drugs.csv', sep=',')

In [178]:
#These are the drugs, with their targets
results.head()

,pert_iname,clinical_phase,moa,target,disease_area,indication
0,"[sar9,met(o2)11]-substance-p",Preclinical,tachykinin antagonist,TACR1,NaN,NaN
0,A-1070722,Preclinical,glycogen synthase kinase inhibitor,GSK3A,NaN,NaN
1,A-1070722,Preclinical,glycogen synthase kinase inhibitor,GSK3B,NaN,NaN
2,A-1120,Preclinical,retinoid receptor ligand,RBP4,NaN,NaN
3,A-317491,Preclinical,purinergic receptor antagonist,P2RX3,NaN,NaN


In [179]:
results.shape

(13097, 6)

In [180]:
d=list(results['pert_iname'].unique())
l_r_d=l_r + d

In [181]:
filt0=np.in1d(results['target'],l_r_d)
results=results[filt0]

In [182]:
results.shape

(2518, 6)

In [183]:
#Selecting all the synonyms for 'activator' and 'inhibitor'
activators=['agonist', 'activator', 'stimulant', 'enhancer', 'reactivator', 'inducer']
inhibitors=['inhibitor', 'antagonist', 'blocker', 'downregulator', 'destabilizer']

In [184]:
#Filtering out the NaNs
fil=~results['moa'].isna()
results=results[fil]

In [185]:
#arrange the drugs in order
results.index=range(len(results.index)) 

In [186]:
#Giving a sign for each drug-target pairs (antagonists: -1, agonists: +1)
results['activator']=0
results['inhibitor']=0
for i in results.index:
    moa=results.loc[i, 'moa']
    is_a=len(set(moa.split()) &set (activators))
    is_i=len(set(moa.split()) &set (inhibitors))
    results.loc[i,['activator', 'inhibitor']]=is_a, is_i

In [187]:
results['sign']=0
for i in results.index:
    if results.loc[i,'activator']==1:
        results.loc[i,'sign']=1
    if results.loc[i,'inhibitor']==1:
        results.loc[i,'sign']=-1

In [188]:
results.head()

,pert_iname,clinical_phase,moa,target,disease_area,indication,activator,inhibitor,sign
0,"[sar9,met(o2)11]-substance-p",Preclinical,tachykinin antagonist,TACR1,NaN,NaN,0,1,-1
1,A-987306,Preclinical,histamine receptor antagonist,AVPR1A,NaN,NaN,0,1,-1
2,A-987306,Preclinical,histamine receptor antagonist,CCR1,NaN,NaN,0,1,-1
3,A-987306,Preclinical,histamine receptor antagonist,HTR1A,NaN,NaN,0,1,-1
4,A-987306,Preclinical,histamine receptor antagonist,HTR1B,NaN,NaN,0,1,-1


In [189]:
results.shape

(2489, 9)

In [30]:
results.to_csv('../results/lincs_drugs_act_inhib.csv', sep=',')

# Importing the gse92742 and gse70138 files

In [190]:
gse92742=pd.read_csv('../data/gse92742/GSE92742_Broad_LINCS_sig_info.txt', sep='\t', header=0, index_col=0) 
gse70138=pd.read_csv('../data/gse70138/GSE70138_Broad_LINCS_sig_info.txt', sep='\t', header=0, index_col=0) 

C:\Users\nidik\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [191]:
gse92742['pert_type'].unique() 

array(['ctl_vehicle', 'trt_cp', 'ctl_untrt', 'trt_sh.cgs',
       'ctl_vehicle.cns', 'ctl_vector.cns', 'ctl_untrt.cns', 'trt_sh.css',
       'trt_lig', 'ctl_vector', 'trt_sh', 'trt_oe', 'trt_oe.mut'],
      dtype=object)

In [192]:
gse70138['pert_type'].unique()

array(['ctl_vehicle', 'trt_cp', 'trt_xpr', 'ctl_untrt', 'ctl_vector'],
      dtype=object)

In [193]:
#filering by perturbation type
good_92742=['trt_sh.cgs','trt_lig','trt_cp','trt_oe']
fil1=np.in1d(gse92742['pert_type'], good_92742)
gse92742=gse92742[fil1]

good_70138=['trt_cp','trt_xpr']
fil2=np.in1d(gse70138['pert_type'], good_70138)
gse70138=gse70138[fil2]

In [194]:
#filtering receptors and ligands
filt1=np.in1d(gse92742['pert_iname'], l_r_d) 
gse92742=gse92742[filt1]

filt2=np.in1d(gse70138['pert_iname'], l_r_d)
gse70138=gse70138[filt2]

In [195]:
#Giving signs for the perturbations:
    #CRIPSR, shRNA : -1 
    #ligand, overexpression: +1
gse70138['sign']=0
gse92742['sign']=0

fil_xpr=gse70138['pert_type']=='trt_xpr'
gse70138.loc[gse70138.index[fil_xpr],'sign']=-1
fil_sh=gse92742['pert_type']=='trt_sh.cgs',
gse92742.loc[gse92742.index[fil_sh],'sign']=-1
fil_lig=gse92742['pert_type']=='trt_lig'
gse92742.loc[gse92742.index[fil_lig],'sign']=+1
fil_oe=gse92742['pert_type']=='trt_oe'
gse92742.loc[gse92742.index[fil_oe],'sign']=+1

In [71]:
gse70138.to_csv('../results/LINCS_gse70138.csv', sep=',')
gse92742.to_csv('../results/LINCS_gse92742.csv', sep=',')

# Creating a seperate file for each perturbation

In [56]:
#CRISPR
fil_xpr2=gse70138['pert_type']=='trt_xpr'
gse70138_trt_xpr=gse70138[fil_xpr2]
gse70138_trt_xpr.to_csv('../results/LINCS_gse70138_trt_xpr.csv', sep=',')

In [57]:
#compound_gse70138
fil_cp1=gse70138['pert_type']=='trt_cp'
gse70138_trt_cp=gse70138[fil_cp1]
gse70138_trt_cp.to_csv('../results/LINCS_gse70138_trt_cp.csv', sep=',')

In [58]:
#compound_gse92742
fil_cp2=gse92742['pert_type']=='trt_cp'
gse92742_trt_cp=gse92742[fil_cp2]
gse92742_trt_cp.to_csv('../results/LINCS_gse92742_trt_cp.csv', sep=',')

In [59]:
#shRNA
fil_sh2=gse92742['pert_type']=='trt_sh.cgs'
gse92742_trt_sh=gse92742[fil_sh2]
gse92742_trt_sh.to_csv('../results/LINCS_gse92742_trt_sh.csv', sep=',')

In [60]:
#ligand
fil_lig2=gse92742['pert_type']=='trt_lig'
gse92742_trt_lig=gse92742[fil_lig2]
gse92742_trt_lig.to_csv('../results/LINCS_gse92742_trt_lig.csv', sep=',')

In [61]:
#over-expression
fil_oe2=gse92742['pert_type']=='trt_oe'
gse92742_trt_oe=gse92742[fil_oe2]
gse92742_trt_oe.to_csv('../results/LINCS_gse92742_trt_oe.csv', sep=',')

In [196]:
#drugs
fil1=np.in1d(gse92742['pert_iname'],results['pert_iname'])
gse92742_drugs=gse92742[fil1]
fil2=np.in1d(gse70138['pert_iname'],results['pert_iname'])
gse70138_drugs=gse70138[fil2]

In [197]:
probe70138=gse70138_drugs.reset_index()
probe92742=gse92742_drugs.reset_index()

In [198]:
#Giving sign to each drug target
gse70138_drug = pd.merge(probe70138,results[['pert_iname','sign']],on='pert_iname', how='left')
gse92742_drug = pd.merge(probe92742,results[['pert_iname','sign']],on='pert_iname', how='left')

In [199]:
gse92742_drug=gse92742_drug.drop_duplicates()
gse70138_drug=gse70138_drug.drop_duplicates()

In [200]:
gse70138_drug=gse70138_drug.set_index('sig_id')
gse92742_drug=gse92742_drug.set_index('sig_id')

In [201]:
gse70138_drug=gse70138_drug.drop(columns=['sign_x'])
gse92742_drug=gse92742_drug.drop(columns=['sign_x'])

In [202]:
gse70138_drug=gse70138_drug.rename(columns={'sign_y':'sign'})
gse92742_drug=gse92742_drug.rename(columns={'sign_y':'sign'})

In [203]:
gse92742_drug.head()

,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_time_unit,pert_itime,distil_id,sign
sig_id,,,,,,,,,,,,
AML001_CD34_24H:BRD-A45664787:0.37037,BRD-A45664787,iloprost,trt_cp,CD34,0.37037,µM,500 nM,24,h,24 h,AML001_CD34_24H_X1_F1B10:J12|AML001_CD34_24H_X...,1
AML001_CD34_24H:BRD-A45664787:1.11111,BRD-A45664787,iloprost,trt_cp,CD34,1.11111,µM,1 µM,24,h,24 h,AML001_CD34_24H_X1_F1B10:J11|AML001_CD34_24H_X...,1
AML001_CD34_24H:BRD-A45664787:10,BRD-A45664787,iloprost,trt_cp,CD34,10,µM,10 µM,24,h,24 h,AML001_CD34_24H_X1_F1B10:I11|AML001_CD34_24H_X...,1
AML001_CD34_24H:BRD-A45664787:3.33333,BRD-A45664787,iloprost,trt_cp,CD34,3.33333,µM,3 µM,24,h,24 h,AML001_CD34_24H_X1_F1B10:I12|AML001_CD34_24H_X...,1
AML001_CD34_24H:BRD-K57391913:0.37037,BRD-K57391913,iloprost,trt_cp,CD34,0.37037,µM,500 nM,24,h,24 h,AML001_CD34_24H_X1_F1B10:F10|AML001_CD34_24H_X...,1


In [45]:
gse92742_drug.to_csv('../results/LINCS_gse92742_drugs.csv', sep=',')
gse70138_drug.to_csv('../results/LINCS_gse70138_drugs.csv', sep=',')

# Creating consensus singatures per perturbations

Consensus signature from drug perturbation on the GSE70138 and GSE92742 datasets

In [221]:
gse70138_drugs=pd.read_csv('../results/LINCS_gse70138_drugs.csv', sep=',',header=0, index_col=0, low_memory=False)
gse92742_drugs=pd.read_csv('../results/LINCS_gse92742_drugs.csv', sep=',',header=0, index_col=0, low_memory=False)

In [222]:
genes_perturbed=list(set(gse70138_drugs['pert_iname']) | set(gse92742_drugs['pert_iname']))
consensus_signatures_trt_drugs=pd.DataFrame(index=genes_perturbed,columns=gene_ids.index.astype(str))
consensus_signatures_trt_drugs.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
fosfosal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JTC-801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
valdecoxib,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
erythromycin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BMS-599626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
for i in range(len(genes_perturbed)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed[i]
    fil1=gse70138_drugs['pert_iname']==gene
    samples=gse70138_drugs.index[fil1]
    if fil1.sum() > 0:
        expression1=parse('../data/gse70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                     cid=samples,rid=gene_ids.index.astype(str)).data_df.T[gene_ids.index.astype(str)]
    else:
        expression1 = None
    fil2=gse92742_drugs['pert_iname']==gene
    samples=gse92742_drugs.index[fil2]
    if fil2.sum() > 0:
        expression2=parse('../data/gse92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                     cid=samples,rid=gene_ids.index.astype(str)).data_df.T[gene_ids.index.astype(str)]
    else:
        expression2 = None
    expression = pd.concat([expression1, expression2])
    consensus_signatures_trt_drugs.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes
Done for 300 genes
Done for 400 genes
Done for 500 genes
Done for 600 genes


In [224]:
consensus_signatures_trt_drugs.columns=gene_ids[consensus_signatures_trt_drugs.columns].values

In [213]:
set(list(consensus_signatures_trt_drugs.index.unique()))&set(list(results.index.unique()))

{"3,3'-diindolylmethane",
 'ABT-239',
 'AG-490',
 'AM-24',
 'AMG-208',
 'ARRY-334543',
 'AST-1306',
 'ATN-161',
 'AV-412',
 'AV-608',
 'BIBU-1361',
 'BIBX-1382',
 'BMS-182874',
 'BMS-536924',
 'BMS-599626',
 'BMS-754807',
 'BMS-777607',
 'BMS-927711',
 'BMY-14802',
 'BMY-7378',
 'BNTX',
 'BP-554',
 'BP-897',
 'BRL-15572',
 'BRL-52537',
 'BRL-54443',
 'BW-723C86',
 'BW-B70C',
 'BX-795',
 'BX-912',
 'CEP-37440',
 'CGP-52411',
 'CGP-53353',
 'CGP-71683',
 'CMPD-1',
 'CP-724714',
 'CP-93129',
 'CP-94253',
 'CPI-1189',
 'CT-7758',
 'CTS-1027',
 'D-4476',
 'DH-97',
 'DPPE',
 'DU-728',
 'DUP-697',
 'E-4031',
 'EMD-1214063',
 'EMD-386088',
 'EMD-66684',
 'ENMD-2076',
 'FIT',
 'FK-866',
 'FK-888',
 'FR-139317',
 'GBR-12935',
 'GR-113808',
 'GR-127935',
 'GR-144053',
 'GR-159897',
 'GTP-14564',
 'GW-441756',
 'GW-5074',
 'GW-583340',
 'HTMT',
 'ICI-162846',
 'ICI-199441',
 'IRL-2500',
 'JLK-6',
 'JNJ-10191584',
 'JNJ-38877605',
 'JTC-801',
 'KF-38789',
 'KI-16425',
 'KI-20227',
 'KI-8751',
 'KW-

In [225]:
consensus_signatures_trt_drugs

,DDR1,PAX8,RPS5,ABCF1,SPAG7,RHOA,RNPS1,SMNDC1,ATP6V0B,RPS6,...,P4HTM,SLC27A3,TBXA2R,RTN2,TSTA3,PPARD,GNA11,WDTC1,PLSCR3,NPEPL1
fosfosal,-0.393281,3.62716,0.391535,-0.888791,-0.639247,0.388546,1.20767,-0.815556,-0.0388766,-1.19985,...,0.0954962,0.168072,-0.580181,-0.61002,0.932762,0.144961,-0.448765,1.5696,0.615967,-0.595085
JTC-801,0.345035,0.321004,0.00113322,0.298489,0.11948,-1.03898,-0.0817776,0.818105,0.266541,-1.06658,...,-0.245301,0.385017,0.393964,0.471965,-0.436718,1.1788,-0.72443,-0.0289182,-0.0554056,0.43961
valdecoxib,0.547191,0.511092,-0.0746301,0.145318,0.505219,0.500257,-0.283213,-0.526816,-0.940512,-0.458733,...,0.345437,0.2623,0.0247299,-0.205452,-0.279761,0.443181,0.111208,0.367318,0.149215,0.266936
erythromycin,0.0596752,0.0659586,0.00193114,-0.0996425,-0.277551,0.170181,0.235633,-0.0782756,-0.393622,-0.0902825,...,0.304055,0.1614,0.14049,-0.472015,-0.0490249,-0.0522405,0.208823,0.121742,0.243679,-0.251227
BMS-599626,0.379269,-0.264654,-0.0857304,-0.429321,-0.368095,0.159735,-0.151,-0.208333,0.0906694,0.0108621,...,0.380933,0.135507,0.0894046,-0.587797,0.210847,0.0930966,0.0224697,0.152358,-0.405571,0.0532778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tirofiban,-0.229952,-0.0831258,-0.442763,-0.444926,-0.288708,0.30761,0.0948675,-0.390662,0.0302178,-0.271827,...,0.057956,-0.11197,-0.0247716,-0.663954,0.246364,-0.105559,-0.186089,0.312565,-0.0400357,0.210398
trimebutine,0.354445,0.274888,-0.331435,0.543945,-0.462552,-0.280125,-0.0603273,-0.10596,-0.740571,-0.267932,...,0.243189,0.955354,1.57422,0.337795,-0.326374,0.00537845,0.154203,0.144607,-0.0134812,-0.062506
methapyrilene,0.397935,0.0360762,0.168133,0.268419,0.121791,-0.298085,0.113569,-0.244147,-0.278672,-0.158213,...,-0.210533,0.146571,0.455082,-0.378915,-0.204159,0.340719,-0.144743,-0.155006,0.269031,-0.101669
duloxetine,0.0280721,0.0274078,0.0199078,-0.342104,-0.142814,0.336882,0.136143,0.0721812,0.0670994,-0.00906922,...,0.350307,-0.343844,-0.322491,-0.601693,0.412237,-0.118532,0.0843156,0.645062,0.129062,-0.23368


In [228]:
results=results.drop_duplicates(subset=['pert_iname'])

In [229]:
results=results.set_index('pert_iname')

In [230]:
fil=np.in1d(list(consensus_signatures_trt_drugs.index.unique()),list(results.index.unique()))
common_drugs=consensus_signatures_trt_drugs[fil]
for i in common_drugs.index:
    if results.loc[i, 'sign']==-1 :
        consensus_signatures_trt_drugs.loc[i]=consensus_signatures_trt_drugs.loc[i].multiply(-1,axis=0, fill_value=None)

In [231]:
consensus_signatures_trt_drugs

,DDR1,PAX8,RPS5,ABCF1,SPAG7,RHOA,RNPS1,SMNDC1,ATP6V0B,RPS6,...,P4HTM,SLC27A3,TBXA2R,RTN2,TSTA3,PPARD,GNA11,WDTC1,PLSCR3,NPEPL1
fosfosal,0.393281,-3.62716,-0.391535,0.888791,0.639247,-0.388546,-1.20767,0.815556,0.0388766,1.19985,...,-0.0954962,-0.168072,0.580181,0.61002,-0.932762,-0.144961,0.448765,-1.5696,-0.615967,0.595085
JTC-801,-0.345035,-0.321004,-0.00113322,-0.298489,-0.11948,1.03898,0.0817776,-0.818105,-0.266541,1.06658,...,0.245301,-0.385017,-0.393964,-0.471965,0.436718,-1.1788,0.72443,0.0289182,0.0554056,-0.43961
valdecoxib,-0.547191,-0.511092,0.0746301,-0.145318,-0.505219,-0.500257,0.283213,0.526816,0.940512,0.458733,...,-0.345437,-0.2623,-0.0247299,0.205452,0.279761,-0.443181,-0.111208,-0.367318,-0.149215,-0.266936
erythromycin,-0.0596752,-0.0659586,-0.00193114,0.0996425,0.277551,-0.170181,-0.235633,0.0782756,0.393622,0.0902825,...,-0.304055,-0.1614,-0.14049,0.472015,0.0490249,0.0522405,-0.208823,-0.121742,-0.243679,0.251227
BMS-599626,-0.379269,0.264654,0.0857304,0.429321,0.368095,-0.159735,0.151,0.208333,-0.0906694,-0.0108621,...,-0.380933,-0.135507,-0.0894046,0.587797,-0.210847,-0.0930966,-0.0224697,-0.152358,0.405571,-0.0532778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tirofiban,0.229952,0.0831258,0.442763,0.444926,0.288708,-0.30761,-0.0948675,0.390662,-0.0302178,0.271827,...,-0.057956,0.11197,0.0247716,0.663954,-0.246364,0.105559,0.186089,-0.312565,0.0400357,-0.210398
trimebutine,0.354445,0.274888,-0.331435,0.543945,-0.462552,-0.280125,-0.0603273,-0.10596,-0.740571,-0.267932,...,0.243189,0.955354,1.57422,0.337795,-0.326374,0.00537845,0.154203,0.144607,-0.0134812,-0.062506
methapyrilene,-0.397935,-0.0360762,-0.168133,-0.268419,-0.121791,0.298085,-0.113569,0.244147,0.278672,0.158213,...,0.210533,-0.146571,-0.455082,0.378915,0.204159,-0.340719,0.144743,0.155006,-0.269031,0.101669
duloxetine,-0.0280721,-0.0274078,-0.0199078,0.342104,0.142814,-0.336882,-0.136143,-0.0721812,-0.0670994,0.00906922,...,-0.350307,0.343844,0.322491,0.601693,-0.412237,0.118532,-0.0843156,-0.645062,-0.129062,0.23368


In [232]:
consensus_signatures_trt_drugs.to_csv('../results/consensus_signature_trt_drugs.csv',sep=',')

Consensus signature from CRISPR perturbation on the GSE70138 dataset

In [88]:
gse70138_trt_xpr=pd.read_csv('../results/LINCS_gse70138_trt_xpr.csv', sep=',',header=0, index_col=0, low_memory=False)

In [89]:
#rows are the perturbed genes, columns are the measured landmark genes from LINCS
genes_perturbed=gse70138_trt_xpr['pert_iname'].unique()
consensus_signatures_gse70138_trt_xpr=pd.DataFrame(index=genes_perturbed,columns=gene_ids.index.astype(str))
consensus_signatures_gse70138_trt_xpr.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
CXCR4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TGFBR1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TGFBR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AXL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
for i in range(len(genes_perturbed)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed[i]
    fil=gse70138_trt_xpr['pert_iname']==gene
    samples=gse70138_trt_xpr.index[fil]
    expression=parse('../data/gse70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                 cid=samples,rid=gene_ids.index.astype(str)).data_df.T[gene_ids.index.astype(str)]
    consensus_signatures_gse70138_trt_xpr.loc[gene]=calc_MODZ(expression)

Done for 0 genes


In [40]:
consensus_signatures_gse70138_trt_xpr.columns=gene_ids[consensus_signatures_gse70138_trt_xpr.columns].values

In [46]:
consensus_signatures_gse70138_trt_xpr=consensus_signatures_gse70138_trt_xpr.multiply(-1, fill_value=None)

In [47]:
consensus_signatures_gse70138_trt_xpr.to_csv('../results/consensus_signature_gse70138_trt_xpr.csv',sep=',')

Consensus signature from cp perturbation on the GSE70138 dataset

In [47]:
gse70138_trt_cp=pd.read_csv('../results/LINCS_gse70138_trt_cp.csv', sep=',',header=0, index_col=0, low_memory=False)

In [48]:
genes_perturbed1=gse70138_trt_cp['pert_iname'].unique()
consensus_signatures_gse70138_trt_cp=pd.DataFrame(index=genes_perturbed1,columns=gene_ids.index)
consensus_signatures_gse70138_trt_cp.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
CP-724714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neratinib,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
crizotinib,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tozasertib,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PD-0325901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
for i in range(len(genes_perturbed1)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed1[i]
    fil=gse70138_trt_cp['pert_iname']==gene
    samples=gse70138_trt_cp.index[fil]
    expression=parse('../data/gse70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse70138_trt_cp.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes
Done for 300 genes
Done for 400 genes
Done for 500 genes
Done for 600 genes
Done for 700 genes
Done for 800 genes
Done for 900 genes
Done for 1000 genes
Done for 1100 genes
Done for 1200 genes
Done for 1300 genes


In [50]:
consensus_signatures_gse70138_trt_cp.columns=gene_ids[consensus_signatures_gse70138_trt_cp.columns].values

In [51]:
consensus_signatures_gse70138_trt_cp.to_csv('../results/consensus_signature_gse70138_trt_cp.csv',sep=',')

Consensus signature from cp perturbation on the GSE92742 dataset

In [52]:
gse92742_trt_cp=pd.read_csv('../results/LINCS_gse92742_trt_cp.csv', sep=',',header=0, index_col=0, low_memory=False)

In [53]:
genes_perturbed2=gse92742_trt_cp['pert_iname'].unique()
consensus_signatures_gse92742_trt_cp=pd.DataFrame(index=genes_perturbed2,columns=gene_ids.index)
consensus_signatures_gse92742_trt_cp.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
trichostatin-a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geldanamycin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iloprost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wortmannin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
calcitriol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
for i in range(len(genes_perturbed2)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed2[i]
    fil=gse92742_trt_cp['pert_iname']==gene
    samples=gse92742_trt_cp.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_cp.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes
Done for 300 genes
Done for 400 genes
Done for 500 genes
Done for 600 genes
Done for 700 genes
Done for 800 genes
Done for 900 genes
Done for 1000 genes
Done for 1100 genes
Done for 1200 genes
Done for 1300 genes
Done for 1400 genes
Done for 1500 genes
Done for 1600 genes
Done for 1700 genes
Done for 1800 genes


In [55]:
consensus_signatures_gse92742_trt_cp.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
trichostatin-a,-0.847422,-0.503865,0.483912,0.132198,-4.53212,0.634433,-0.698595,-1.79651,1.22057,-0.768776,...,-2.1688,-0.0347143,2.49821,1.53952,-0.665192,3.59326,1.76075,2.27205,0.0706258,-1.3203
geldanamycin,0.542107,0.291332,-0.0559945,0.232725,1.71355,-0.569987,-0.462787,1.32345,-1.24771,-0.341131,...,0.31085,0.235081,0.410122,0.351053,-1.19802,0.734019,-0.0621831,0.679144,-0.185019,0.608295
iloprost,-0.145694,0.157691,0.0225018,-0.261297,-0.333339,-0.146206,-0.101394,0.0563712,0.00170397,-0.124097,...,0.182753,-0.176382,-0.0254652,-0.331386,0.104815,0.103503,0.222015,-0.0912689,-0.128151,-0.0233869
wortmannin,1.05525,0.461242,-0.0889274,0.0519562,0.647678,-0.272643,-0.342935,-0.397896,-1.00563,0.143087,...,0.401039,0.440965,0.483755,0.574008,-0.675819,0.927863,-0.383918,1.2194,0.524388,0.736217
calcitriol,0.342617,0.389063,-0.0431887,-0.00860405,0.00678472,0.179245,-0.33049,-0.290733,-0.395502,-0.206041,...,0.146251,-0.00878185,0.195036,-0.208913,-0.0821654,0.0700581,0.200104,-0.00942942,-0.0878858,0.11443


In [56]:
consensus_signatures_gse92742_trt_cp.columns=gene_ids[consensus_signatures_gse92742_trt_cp.columns].values

In [57]:
consensus_signatures_gse92742_trt_cp.to_csv('../results/consensus_signature_gse92742_trt_cp.csv',sep=',')

Consensus signature from shRNA perturbation on the GSE92742 dataset

In [48]:
gse92742_trt_sh=pd.read_csv('../results/LINCS_gse92742_trt_sh.csv', sep=',',header=0, index_col=0, low_memory=False)

In [49]:
genes_perturbed3=gse92742_trt_sh['pert_iname'].unique()
consensus_signatures_gse92742_trt_sh=pd.DataFrame(index=genes_perturbed3,columns=gene_ids.index)
consensus_signatures_gse92742_trt_sh.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACVR1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACVR1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
for i in range(len(genes_perturbed3)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed3[i]
    fil=gse92742_trt_sh['pert_iname']==gene
    samples=gse92742_trt_sh.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_sh.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes
Done for 300 genes


In [51]:
consensus_signatures_gse92742_trt_sh.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
A2M,-0.370345,-0.0211347,0.446688,-0.631813,-0.688607,0.517196,0.351272,0.00260253,-0.0165566,-0.00810191,...,0.20102,-0.683747,-0.377338,-1.14508,0.336994,-0.345827,0.363675,0.0479457,0.455031,-0.784672
ACVR1,-0.364293,0.230737,0.435273,-0.206998,0.572639,0.829783,0.332642,-0.0163975,-0.367624,0.0408233,...,-0.139032,-0.223809,-0.37808,-0.715195,0.150829,-0.308099,0.456004,0.0397732,-0.262659,-0.608555
ACVR1B,-0.0212503,0.731236,0.0945946,0.455959,-0.266212,0.305241,0.469773,-0.552765,-0.285593,0.0158941,...,1.01315,-0.403799,0.377017,-0.369512,0.267321,0.206183,0.662095,-0.198008,0.523423,0.288574
ADAM15,0.373369,0.271718,-0.278022,-0.30926,-0.244253,-0.351224,-0.589339,-0.107752,-0.520766,-0.259483,...,0.473877,0.329349,-0.101799,0.133489,-0.6313,0.60008,-0.0498236,0.461169,0.466588,-0.0117425
ADAM17,-0.0853006,0.20938,0.1143,-0.0883846,-0.0908374,-0.0278078,-0.789735,-0.11546,-0.559442,-0.268747,...,0.0051903,0.0969632,0.389772,0.245823,-0.396961,0.635162,0.117535,-0.0789634,0.421095,0.1974


In [52]:
consensus_signatures_gse92742_trt_sh.columns=gene_ids[consensus_signatures_gse92742_trt_sh.columns].values

In [53]:
consensus_signatures_gse92742_trt_sh=consensus_signatures_gse92742_trt_sh.multiply(-1, fill_value=None)

In [54]:
consensus_signatures_gse92742_trt_sh.to_csv('../results/consensus_signature_gse92742_trt_sh.csv',sep=',')

Consensus signature from shRNA perturbation filtered by gene expression on the GSE92742 dataset

In [55]:
gse92742_trt_sh_fil=pd.read_csv('../results/LINCS_gse92742_trt_sh_fil.csv', sep=',',header=0, index_col=0, low_memory=False)

In [56]:
genes_perturbed7=gse92742_trt_sh_fil['pert_iname'].unique()
consensus_signatures_gse92742_trt_sh_fil=pd.DataFrame(index=genes_perturbed7,columns=gene_ids.index)
consensus_signatures_gse92742_trt_sh_fil.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACVR1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACVR1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADAM17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
for i in range(len(genes_perturbed7)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed7[i]
    fil=gse92742_trt_sh_fil['pert_iname']==gene
    samples=gse92742_trt_sh_fil.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_sh_fil.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes


In [58]:
consensus_signatures_gse92742_trt_sh_fil.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
A2M,-0.514138,0.00927261,0.0179969,-0.685129,-0.995319,0.0425183,0.57063,-0.285279,-0.0584261,-0.0695455,...,0.273541,-0.449426,-0.0785917,-0.562822,0.128081,-0.14653,0.390276,0.780865,0.717586,-0.360812
ACVR1,-0.364293,0.230737,0.435273,-0.206998,0.572639,0.829783,0.332642,-0.0163975,-0.367624,0.0408233,...,-0.139032,-0.223809,-0.37808,-0.715195,0.150829,-0.308099,0.456004,0.0397732,-0.262659,-0.608555
ACVR1B,-0.0212503,0.731236,0.0945946,0.455959,-0.266212,0.305241,0.469773,-0.552765,-0.285593,0.0158941,...,1.01315,-0.403799,0.377017,-0.369512,0.267321,0.206183,0.662095,-0.198008,0.523423,0.288574
ADAM15,0.373369,0.271718,-0.278022,-0.30926,-0.244253,-0.351224,-0.589339,-0.107752,-0.520766,-0.259483,...,0.473877,0.329349,-0.101799,0.133489,-0.6313,0.60008,-0.0498236,0.461169,0.466588,-0.0117425
ADAM17,-0.0853006,0.20938,0.1143,-0.0883846,-0.0908374,-0.0278078,-0.789735,-0.11546,-0.559442,-0.268747,...,0.0051903,0.0969632,0.389772,0.245823,-0.396961,0.635162,0.117535,-0.0789634,0.421095,0.1974


In [59]:
consensus_signatures_gse92742_trt_sh_fil.columns=gene_ids[consensus_signatures_gse92742_trt_sh_fil.columns].values

In [60]:
consensus_signatures_gse92742_trt_sh_fil=consensus_signatures_gse92742_trt_sh_fil.multiply(-1, fill_value=None)

In [61]:
consensus_signatures_gse92742_trt_sh_fil.to_csv('../results/consensus_signature_gse92742_trt_sh_fil.csv',sep=',')

Consensus signature from ligand perturbation on the GSE92742 dataset

In [71]:
gse92742_trt_lig=pd.read_csv('../results/LINCS_gse92742_trt_lig.csv', sep=',',header=0, index_col=0, low_memory=False)

In [72]:
genes_perturbed4=gse92742_trt_lig['pert_iname'].unique()
consensus_signatures_gse92742_trt_lig=pd.DataFrame(index=genes_perturbed4,columns=gene_ids.index)
consensus_signatures_gse92742_trt_lig.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
ADM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AGT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANGPT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANGPT2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANGPT4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
for i in range(len(genes_perturbed4)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed4[i]
    fil=gse92742_trt_lig['pert_iname']==gene
    samples=gse92742_trt_lig.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_lig.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes


In [74]:
consensus_signatures_gse92742_trt_lig.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
ADM,-0.311746,-0.598979,0.279604,-0.713548,-1.76606,0.331375,0.0142727,-1.40513,0.50739,-0.364242,...,0.206009,-0.299599,0.460008,0.17011,-0.0937956,1.51991,0.853947,0.960275,-0.0268266,-0.0742602
AGT,0.328147,0.187768,-0.214431,0.38774,0.104206,0.686732,-0.234371,-0.572312,-0.654196,-0.126784,...,0.335688,0.0188236,0.412239,0.204362,-0.674465,0.261729,0.165706,0.20852,-0.264325,0.505703
ANGPT1,-0.29077,0.331472,0.0413283,-0.156881,-0.0400801,0.0208699,0.246865,-0.197276,0.0766549,-0.342726,...,0.679564,-0.21031,-0.274827,-0.4247,0.255042,-0.299798,0.43241,0.37303,0.701792,-0.419092
ANGPT2,0.681153,-0.265093,0.158823,0.164295,0.143002,-0.0873656,-0.383005,-0.519193,-0.784639,-0.210277,...,-0.0548271,0.636016,1.14714,0.357248,-0.728819,0.517119,0.24571,-0.0192953,0.243718,0.500907
ANGPT4,-0.105004,0.168645,-0.413267,-0.507766,-0.695529,-0.302776,-0.775928,0.737322,-1.26081,-1.46682,...,-0.40731,0.866114,1.03953,0.273502,-1.19526,0.991168,-0.241819,0.144145,-0.0720215,0.473727


In [75]:
consensus_signatures_gse92742_trt_lig.columns=gene_ids[consensus_signatures_gse92742_trt_lig.columns].values

In [76]:
consensus_signatures_gse92742_trt_lig.to_csv('../results/consensus_signature_gse92742_trt_lig.csv',sep=',')

Consensus signature from overexpression perturbation on the GSE92742 dataset

In [77]:
gse92742_trt_oe=pd.read_csv('../results/LINCS_gse92742_trt_oe.csv', sep=',',header=0, index_col=0, low_memory=False)

In [78]:
genes_perturbed5=gse92742_trt_oe['pert_iname'].unique()
consensus_signatures_gse92742_trt_oe=pd.DataFrame(index=genes_perturbed5,columns=gene_ids.index)
consensus_signatures_gse92742_trt_oe.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
ACVR1B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACVR1C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMHR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AXL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DDR2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
for i in range(len(genes_perturbed5)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed5[i]
    fil=gse92742_trt_oe['pert_iname']==gene
    samples=gse92742_trt_oe.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_gse92742_trt_oe.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes


In [80]:
consensus_signatures_gse92742_trt_oe.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
ACVR1B,-0.213152,-0.156863,-0.0448605,0.648634,-1.4457,-0.174522,-0.585348,0.405451,0.055252,0,...,0.703523,0.217143,1.74089,-0.313028,0.893814,0.322672,0.719431,-0.778168,0.170625,-0.301017
ACVR1C,1.28437,0.974051,-0.21175,0.108015,-0.278265,-0.570205,-0.627374,0.305463,-0.869867,0,...,-0.873698,0.631,-0.0798211,0.583505,-0.253972,1.49154,0.133039,0.363452,-0.44913,0.763833
AMHR2,1.19438,0.122225,-0.3159,0.350275,0.2457,-0.6482,0.4137,-0.632,-0.108825,0,...,0.189425,0.149825,0.029275,0.642825,0.4373,0.7436,0.232775,-0.02445,0.008475,-0.662775
AXL,1.09185,-0.361738,-0.078832,0.959603,-0.286295,-0.166586,-0.578821,0.468852,-0.434471,-0.723212,...,-0.684398,1.13625,0.689128,0.958076,-0.828212,-0.357352,-0.252046,-0.635741,-0.614348,-0.0437426
DDR2,-0.266142,-0.506923,0.0856334,-0.120144,-0.163612,0.30325,0.978982,-0.523146,0.63971,-0.316952,...,2.68877,-0.149931,-0.631309,-1.335,0.588573,-0.960427,0.426914,1.10355,-0.322558,-0.863717


In [81]:
consensus_signatures_gse92742_trt_oe.columns=gene_ids[consensus_signatures_gse92742_trt_oe.columns].values

In [82]:
consensus_signatures_gse92742_trt_oe.to_csv('../results/consensus_signature_gse92742_trt_oe.csv',sep=',')